# LLM Application Developer Programme (LADP)

<h2>Phase 2: Project Phase<br>
Knowledge Transfer 3: Hands on with Llama index, OpenAI function calling, and low code tools


**This notebook is the property of AI Singapore® and may not be distributed or sold without permission.**


**Outline of Knowledge Transfer 3**

1. Hands-on RAG with LlamaIndex
2. OpenAI function calling
3. Further learning


---


# Instructions for participant

1. Make a copy of the Knowledge Transfer 3 notebook from the shared folder.

2. You need the **.env** file which you used during the Deepskilling Phase. Upload the file in the notebook's file structure before you begin.

3. From the [LADP Knowledge Transfer data folder](https://drive.google.com/drive/folders/123gTWsKDW7FKSQu-H_j3LhBrBGhMagyW?usp=sharing), download the 3 files:

- **aiml-learning-resources.csv** (This file was saved from a spreadsheet containing a dataset of AI/ML learning resources. You may open up the CSV file with a spreadsheet programme to see what it contains. Note: CSV stands for "comma separated values" -- basically a text file containing spreadsheet data without any special formatting.)
- **barbie.pdf** (PDF file created from Wikipedia entry on the recent _Barbie_ (2023) movie.)
- **oppenheimer.pdf** (PDF file created from Wikipedia entry on the recent _Oppenheimer_ (2023) movie.)

4. In your notebook's file structure, create a new folder called “data”. Upload the 3 files into the “data” folder.

5. Run the Setup Code.


---


# 1. Hands-on RAG with LlamaIndex

What we have done so far with LangChain can also be done with LlamaIndex. Below, we demonstrate some coding recipes for you to try.

LlamaIndex has its own built-in vector store (a.k.a. vector database), which can run in memory or on disk. LlamaIndex can also integrate with vector databases such as Chroma and others.

For simplicity, we will be using the built-in vector store in LlamaIndex for most of the demos below. In the last example, we will try out Chroma vector database.


## A note about nodes

In LlamaIndex, "nodes" are the smallest unit of content, so that means **chunks** are called **nodes** in LlamaIndex.


## A note about embedding models

With our LlamaIndex hands-on exercises, we will change the embedding model from "text-embedding-ada-002" to "BAAI/bge-base-en-v1.5".

If you want to switch from one model to another, here is the relevant code for your reference:

**text-embedding-ada-002**

```python
from llama_index import LangchainEmbedding
# Define embedding model:
embedding = LangchainEmbedding(
    OpenAIEmbeddings(model="text-embedding-ada-002")
)
```

**BAAI/bge-base-en-v1.5**

```python
from llama_index.embeddings import HuggingFaceEmbedding
# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
```


## Run setup code

Run this code cell to make sure the AzureOpenAI API is working:


In [ ]:
# Install software packages:
%pip install openai==0.28.1
%pip install llama-index==0.8.61
%pip install transformers==4.35.0
%pip install python-dotenv==1.0.0
%pip install chromadb==0.4.15
%pip install pypdf==3.17.0

# Import modules:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms import AzureOpenAI

# Import display/formatting modules:
from IPython.display import Markdown, display

# Import environment modules:
from dotenv import load_dotenv
import os

# Load .env file and get API key, API base and deployment name:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

# Instantiate an LLM instance:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Instantiate a prompt and get the response:
response = llm.complete("The sky is a beautiful blue and")
print(response)

## Get embeddings with LlamaIndex

Let's use LlamaIndex's `HuggingFaceEmbedding` module to generate embedding vectors for a short sentence. For instructions, refer to the [LlamaIndex documentation](https://docs.llamaindex.ai/en/latest/module_guides/models/embeddings.html#standalone-usage).

For our LlamaIndex hands-on exercises, we will use **"BAAI/bge-base-en-v1.5"** hosted by Hugging Face Hub. This is an open-source model which you don't have to pay to use.

Notice that "BAAI/bge-base-en-v1.5" has **768 dimensions**. Different embedding models have different number of dimensions.

When we print out the actual embedding vector, notice that it is a very long output.


In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Encode embedding vectors for a short sentence:
embeddings = embedding.get_text_embedding(
    "I love working with large language models."
)

# See embedding details:
print('Length of embedding vector (number of dimensions):', len(embeddings))
print('Actual embedding vector:', embeddings)

## LlamaIndex RAG Q&A using in-memory VectorStoreIndex

In this exercise, we will be using LlamaIndex to create a RAG application:

- Loading multiple files in a directory.
- Using LlamaIndex's built-in VectorStoreIndex, running in-memory (RAM). By default, if you don't tell VectorStoreIndex to save on-disk to a local directory, then it will just run in-memory.
- Showing citations which were used to inform the LLM's completion. Note that by default, LlamaIndex will retrieve the **top 2 search results**, so you should expect to get 2 citations.

The code cell below will do all these, but there are 2 missing blocks in the code which you need to complete for it to work:

**Directory reader for loading the files**

You can use the following code to use SimpleDirectoryReader to load multiple files into the LlamaIndex pipeline. Remember to update the string value of your data folder name. It's the directory you created for uploading the PDF and CSV files.

```python
# Import SimpleDirectoryReader module:
from llama_index import SimpleDirectoryReader
# Load in documents:
documents = SimpleDirectoryReader('____').load_data()
```

**Instantiate the vector store index**

After you have loaded the files into the `documents` variable, we will need to pass `documents` into the `VectorStoreIndex` to construct a vector index. You can do this with one line of code:

```python
index = VectorStoreIndex.from_documents(documents, show_progress=True)
```

Note that the argument `show_progress=True` will show some progress indicators when constructing the index.

Add in the 2 missing blocks in the code and then run the code cell. Notice the number of citations that are given to the LLM to form its response.


In [ ]:
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import LangchainEmbedding
from llama_index.text_splitter import TokenTextSplitter

# Load multiple files by using a directory reader:
#________________
#________________
#________________
#________________


# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
  chunk_size=512,
  chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Instantiate the vector store index:
#______________________________________________

# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query("In which year were the Barbie movie and Oppenheimer movie released?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

#### Code Activity Solution:

After completing the above code activity, you may click on "Show code" to reveal the Code Activity Solution below for comparison.

Alternatively, if you are stuck, you may also refer to the Code Activity Solution below for comparison.

In [2]:
# @title
# Solution
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import LangchainEmbedding
from llama_index.text_splitter import TokenTextSplitter

# Load multiple files by using a directory reader:
# REPLACE '________________'

# Import SimpleDirectoryReader module:
from llama_index import SimpleDirectoryReader
# Load in documents:
documents = SimpleDirectoryReader('data').load_data()


# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Instantiate the vector store index:
# REPLACE '______________________________________________'

index = VectorStoreIndex.from_documents(documents, show_progress=True)

# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query(
    "In which year were the Barbie movie and Oppenheimer movie released?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Parsing documents into nodes:   0%|          | 0/14 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/252 [00:00<?, ?it/s]

The Barbie movie and Oppenheimer movie were both released in the year 2023.

Citations:

> Source (Doc id: 756b58ef-a982-40ac-ba94-276ece615066): Oppenheimer (film)
Oppenheimer is a 2023 biographical drama film written and directed by Christopher Nolan. Based 
on the 2005 biography American Prometheus by Kai Bird and Martin J. Sherwin, the film 
chronicles the life of American theoretical physicist and polymath J. Robert Oppenheimer. The 
story predominantly focuses on Oppenheimer's early studies, his direction of the Manhattan Project 
during World War II, and his eventual fall from grace due to his 1954 security hearing; juxtaposed 
with this are events surrounding Oppenheimer's relationship with Lewis Strauss, a senior member 
of the United States Atomic Energy Commission who sees Oppenheimer as a rival. It stars Cillian 
Murphy as Oppenheimer and Robert Downey Jr. as Strauss, with the remaining ensemble 
supporting cast including Emily Blunt, Matt Damon, Florence Pugh, Josh Hartnet

### A note about ServiceContext

In the code above and below, there is a section for configuring the service context.

`ServiceContext` is a bundle of commonly used resources, such as LLM and embedding model, used during the indexing and querying stage in a LlamaIndex pipeline. You bundle them together in one place under `ServiceContext` so that you don't have to specify them over and over again in multiple places.

Refer to the [LlamaIndex documentation](https://docs.llamaindex.ai/en/latest/core_modules/supporting_modules/service_context.html) to learn more.


## LlamaIndex using VectorStoreIndex on disk

In the above cell, by default the vector index is stored in memory. This means the index will not persist after the session is closed.

To avoid having to rebuild the vector index in every session, we need to persist it across sessions by saving it to disk.

In the code below, there is a missing section under "Disk storage for vector index".

You can use the code below, but remember to fill in the string value for the directory to be used for storing the vector index. You can name it anything you want, such as `"vstore"` or `"vdb"`.

```python
# Define storage location folder:
VECTORSTORE = '__________'
# Load the vector index from local folder if exists, else build it from scratch.
if os.path.exists(VECTORSTORE):
    storage_context = StorageContext.from_defaults(
        persist_dir=VECTORSTORE
    )
    index = load_index_from_storage(storage_context)
else:
    index = VectorStoreIndex.from_documents(documents, show_progress=True)

    # Save the index to disk so that we don't have to rebuild it every time:
    index.storage_context.persist(persist_dir=VECTORSTORE)
```

After adding in the code, run it to see if it works. Notice that under your "Files" sidebar, there is now a new folder which you named.


In [ ]:
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from langchain.embeddings import OpenAIEmbeddings
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.text_splitter import TokenTextSplitter

# Import SimpleDirectoryReader module:
from llama_index import SimpleDirectoryReader
# Load in documents:
documents = SimpleDirectoryReader('data').load_data()

# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
  chunk_size=512,
  chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Disk storage for vector indexes.
#____________
#____________
#____________
#____________
#____________

# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query("In which year were the Barbie movie and Oppenheimer movie released?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

#### Code Activity Solution:

After completing the above code activity, you may click on "Show code" to reveal the Code Activity Solution below for comparison.

Alternatively, if you are stuck, you may also refer to the Code Activity Solution below for comparison.

In [ ]:
# @title
# Solution
# Import modules:
from llama_index import load_index_from_storage
from llama_index.storage.storage_context import StorageContext
#################################################################
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from langchain.embeddings import OpenAIEmbeddings
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.text_splitter import TokenTextSplitter

# Import SimpleDirectoryReader module:
from llama_index import SimpleDirectoryReader
# Load in documents:
documents = SimpleDirectoryReader('data').load_data()

# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Disk storage for vector indexes.
# REPLACE '____________'

# Define storage location folder:
VECTORSTORE = 'vdb'
# Load the vector index from local folder if exists, else build it from scratch.
if os.path.exists(VECTORSTORE):
    storage_context = StorageContext.from_defaults(
        persist_dir=VECTORSTORE
    )
    index = load_index_from_storage(storage_context)
else:
    index = VectorStoreIndex.from_documents(documents, show_progress=True)

    # Save the index to disk so that we don't have to rebuild it every time:
    index.storage_context.persist(persist_dir=VECTORSTORE)

# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query(
    "In which year were the Barbie movie and Oppenheimer movie released?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

## LlamaIndex using CSV file

In the previous exercise, we loaded multiple files from a directory by using the `SimpleDirectoryReader`:

```python
# Import SimpleDirectoryReader module:
from llama_index import SimpleDirectoryReader
# Load in documents:
documents = SimpleDirectoryReader('data').load_data()
```

LlamaIndex offers several "readers" for loading data and parsing them into "nodes" a.k.a. chunks. To learn more, [read this guide](https://docs.llamaindex.ai/en/stable/understanding/loading/loading.html).

For loading a CSV file, we can use PagedCSVLoader from [LlamaHub](https://llamahub.ai/). PagedCSVLoader formats each row in an LLM-friendly way and inserts it into a separate Document (a.k.a. "node" or chunk).

We can replace the `SimpleDirectoryReader` code block with the `PagedCSVLoader` code below:

```python
# Download PagedCSVReader from LlamaHub:
PagedCSVReader = download_loader("PagedCSVReader")
# Instantiate loader:
loader = PagedCSVReader(encoding="utf-8")
# Load in CSV file:
documents = loader.load_data(file=Path('data/aiml-learning-resources.csv'))
```

In the code below, replace the `SimpleDirectoryReader` code block with the `PagedCSVLoader` code block and then run the code to get a completion from the LLM:


In [ ]:
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import LangchainEmbedding
from llama_index.text_splitter import TokenTextSplitter
from pathlib import Path
from llama_index import download_loader

# Import SimpleDirectoryReader module:
from llama_index import SimpleDirectoryReader
# Load in documents:
documents = SimpleDirectoryReader('data').load_data()

# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
  chunk_size=512,
  chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Construct vector index:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query("Does Mathematics for Machine Learning cover any advanced machine learning techniques?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

#### Code Activity Solution:

After completing the above code activity, you may click on "Show code" to reveal the Code Activity Solution below for comparison.

Alternatively, if you are stuck, you may also refer to the Code Activity Solution below for comparison.

In [ ]:
# @title
# Solution
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import LangchainEmbedding
from llama_index.text_splitter import TokenTextSplitter
from pathlib import Path
from llama_index import download_loader

# # Import SimpleDirectoryReader module:
# from llama_index import SimpleDirectoryReader
# # Load in documents:
# documents = SimpleDirectoryReader('data').load_data()
#######################################################
# Download PagedCSVReader from LlamaHub:
PagedCSVReader = download_loader("PagedCSVReader")
# Instantiate loader:
loader = PagedCSVReader(encoding="utf-8")
# Load in CSV file:
documents = loader.load_data(file=Path('data/aiml-learning-resources.csv'))

# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Construct vector index:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query(
    "Does Mathematics for Machine Learning cover any advanced machine learning techniques?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

In [ ]:
# View the prompts that are available here
# Commonly Used Prompts
# The most commonly used prompts will be the text_qa_template and the refine_template.
# text_qa_template - used to get an initial answer to a query using retrieved nodes
# refine_template - used when the retrieved text does not fit into a single LLM call with response_mode="compact" (the default), or when more than one node is retrieved using response_mode="refine". The answer from the first query is inserted as an existing_answer, and the LLM must update or repeat the existing answer based on the new context.

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))


prompts_dict = query_engine.get_prompts()


display_prompt_dict(prompts_dict)

## LlamaIndex: Top-k + MMR mode

By default, the LlamaIndex query engine retrieves the top 2 search results when you run a query on the document(s) to be searched. (You might have noticed this in the previous examples when we printed out the citations.) This number can be adjusted.

To prepare you for the next exercise, look back to the previous exercise and find the line of code which instantiates a query engine with default settings. We are going to modify it.

The code below is a copy of the previous exercise. We are going to modify it. Instead of instantiating a **query engine** with default settings, we want to install something like a **control panel** on the query engine, which allows us to adjust some settings -- such as the top-k number. This control panel is called **VectorIndexRetriever**.

So, we will instantiate and configure a new **VectorIndexRetriever** (the control panel which allows us to set a **similarity_top_k** argument), and then we will pass this VectorIndexRetriever to the query engine. This will allow us to adjust the top-k number.

The VectorIndexRetriever will also allow us to activate **MMR mode**. MMR stands for Maximum Marginal Relevance and gives more diversity in the search results for better retrievals.

**Step 1**

This is the code to instantiate and configure a VectorIndexRetriever. Find the appropriate spot in the code below and insert this code:

```python
# Instantiate and configure a VectorIndexRetriever:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
    vector_store_query_mode="mmr",
    vector_store_kwargs={"mmr_threshold": 0.2}
)
```

Notice that we now have top-k and MMR. Note the new top-k value of 3 is different from the default number of search results which is 2.

**Step 2**

Next, look for the line of code which instantiates a query engine with default settings. Replace it with this code which instantiates a **RetrieverQueryEngine**:

```python
# Instantiate RetrieverQueryEngine and pass in the VectorIndexRetriever:
query_engine = RetrieverQueryEngine(
    retriever=retriever
)
```

When you are done, run the code. Check how many search results you get.


In [ ]:
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import LangchainEmbedding
from llama_index.text_splitter import TokenTextSplitter
from pathlib import Path
from llama_index import download_loader
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

# Load in documents:
PagedCSVReader = download_loader("PagedCSVReader")
loader = PagedCSVReader(encoding="utf-8")
documents = loader.load_data(file=Path('data/aiml-learning-resources.csv'))

# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
  chunk_size=512,
  chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Construct vector index:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

# Instantiate and configure a VectorIndexRetriever:
# ____________________

# REPLACE THIS CODE BLOCK WITH NEW CODE TO INSTANTIATE A RetrieverQueryEngine.
# Instantiate a query engine with default settings:
query_engine = index.as_query_engine()

# Query the index:
response = query_engine.query("Does Mathematics for Machine Learning cover any advanced machine learning techniques?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

#### Code Activity Solution:

After completing the above code activity, you may click on "Show code" to reveal the Code Activity Solution below for comparison.

Alternatively, if you are stuck, you may also refer to the Code Activity Solution below for comparison.

In [ ]:
# @title
# Solution
# Import modules:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import AzureOpenAI
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context
)
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import LangchainEmbedding
from llama_index.text_splitter import TokenTextSplitter
from pathlib import Path
from llama_index import download_loader
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

# Load in documents:
PagedCSVReader = download_loader("PagedCSVReader")
loader = PagedCSVReader(encoding="utf-8")
documents = loader.load_data(file=Path('data/aiml-learning-resources.csv'))

# Instantiate a text splitter for the node parser:
text_splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
)

# Chunking: Instantiate a node parser to parse the documents into Nodes:
parser = SimpleNodeParser(text_splitter=text_splitter)
nodes = parser.get_nodes_from_documents(documents)

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Configure service context:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
set_global_service_context(service_context)

# Construct vector index:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

# Instantiate and configure a VectorIndexRetriever:
# ____________________
# Instantiate and configure a VectorIndexRetriever:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
    vector_store_query_mode="mmr",
    vector_store_kwargs={"mmr_threshold": 0.2}
)

# # REPLACE THIS CODE BLOCK WITH NEW CODE TO INSTANTIATE A RetrieverQueryEngine.
# # Instantiate a query engine with default settings:
# query_engine = index.as_query_engine()

# Instantiate RetrieverQueryEngine and pass in the VectorIndexRetriever:
query_engine = RetrieverQueryEngine(
    retriever=retriever
)

# Query the index:
response = query_engine.query(
    "Does Mathematics for Machine Learning cover any advanced machine learning techniques?")
print(response)
print('\nCitations:\n')
print(response.get_formatted_sources(length=5000))

Notice that in our code above, we set "mmr_threshold" to 0.2.

**Mini Quiz**

1. Why is MMR useful?
2. What is mmr_threshold? What does it do? Hint: https://docs.llamaindex.ai/en/stable/examples/vector_stores/SimpleIndexDemoMMR.html


## LlamaIndex with ChromaDB

So far in our coding exercises, we have been using LlamaIndex's own built-in vector database. This is convenient but optional, because LlamaIndex can integrate with [several external vector databases](https://docs.llamaindex.ai/en/stable/community/integrations/vector_stores.html) (click on the link to see the full list).

One of the most popular vector databases is [Chroma](https://www.trychroma.com/), due to its simplicity, flexibility and speed. You can deploy it on the cloud or on premise, and it can handle multiple data types.

Below is some code showing how to integrate LlamaIndex with Chroma, running it in memory.

Go ahead and run this code cell:


In [ ]:
# Import modules:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
import chromadb

# # Instantiate Chroma client:
client = chromadb.Client()


# Set up Chroma collection:
chroma_collection = client.get_or_create_collection("collection1")

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Load text documents from directory:
documents = SimpleDirectoryReader('data').load_data()

# Set up ChromaVectorStore and load in data:
vector_store = ChromaVectorStore(
    chroma_collection=chroma_collection
)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

# Query Data
query_engine = index.as_query_engine()
response = query_engine.query(
    "In which year were the Barbie movie and Oppenheimer movie released?")
display(Markdown(f"{response}"))

In the example code above, we ran Chroma in memory, which means the data will not survive between sessions. To avoid rebuilding the index for every session, we want to be able to save it to disk. We can easily do this with one line of code:

```python
# Instantiate Chroma client:
client = chromadb.PersistentClient(path="chroma_db")
```

Refer to the [LlamaIndex documentation on Chroma](https://docs.llamaindex.ai/en/stable/examples/vector_stores/ChromaIndexDemo.html) and [Chroma documentation](https://docs.trychroma.com/) to learn more.

In the code cell below, try copying the code and modify it to save to disk. If you run it, you should then see the new directory appear in your folder sidebar.


#### Code Activity Solution:

After completing the above code activity, you may click on "Show code" to reveal the Code Activity Solution below for comparison.

Alternatively, if you are stuck, you may also refer to the Code Activity Solution below for comparison.

In [ ]:
# @title
# Solution
# Import modules:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
import chromadb

# # Instantiate Chroma client:
# client = chromadb.Client()
# Instantiate Chroma client:
client = chromadb.PersistentClient(path="chroma_db")

# Set up Chroma collection:
chroma_collection = client.get_or_create_collection("collection1")

# Define embedding model:
embedding = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate an LLM:
llm = AzureOpenAI(
    engine="gpt-35-turbo",
    model="gpt-35-turbo",
    temperature=0
)

# Load text documents from directory:
documents = SimpleDirectoryReader('data').load_data()

# Set up ChromaVectorStore and load in data:
vector_store = ChromaVectorStore(
    chroma_collection=chroma_collection
)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding
)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

# Query Data
query_engine = index.as_query_engine()
response = query_engine.query(
    "In which year were the Barbie movie and Oppenheimer movie released?")
display(Markdown(f"{response}"))

---


# 2. OpenAI function calling


<center>
<img src='data:image/png;base64,
...
' /><br>
Function calling is structurally similar to Retrieval Augmented Generation.
</center>


If you look at the diagram above showing an example of function calling, you will notice that it is structurally similar to Retrieval Augmented Generation.

The only difference is the first step where the extra knowledge is obtained by calling a function which returns the knowledge, instead of retrieving some chunks through information retrieval using a vector database.


## Motivation

You may remember during the previous Knowledge Transfer 1, we tried to get the LLM to extract some information from the user query, and output it in JSON format.

The user query was:

```
Can you recommend any books or courses about python and machine learning?
```

The LLM's initial output was:

```
[{'Resource type': 'Online course'}, {'Learning area': 'Python programming'}, {'Learning area': 'Machine Learning'}]
```

Which was **incorrect**. It should have included the "Ebook" resource type, and the learning area should have been one list instead of two separate dicts. We needed to do some prompt engineering to get the correct output, which is this:

```
[{'Resource type': ['Ebook', 'Online course'], 'Learning area': ['Python programming', 'Machine Learning']}]
```

This kind of inconsistent output when it comes to JSON format has become a major annoyance for LLM developers. It breaks things when we want to use that JSON output for some downstream task.

Another common scenario is that the LLM might output some explanatory text along with the JSON string. This kind of output will trigger an error if you try to parse it directly. As developers, we would have to use prompt engineering to steer the LLM output to make it cleaner, or else use regular expressions to parse the output to make it useable.

Thankfully, OpenAI has a new feature called **"function calling"**, which is meant to address this situation.


<center>
<img src='data:image/jpeg;base64,
...
' /><br>
OpenAI announced in June 2023 that its API can now do function calling.<br>
Source: "<a href="https://openai.com/blog/function-calling-and-other-api-updates">OpenAI</a>".
</center>


## What is OpenAI function calling?

Function calling provides one or more function descriptions as input for GPT-3.5 and GPT-4, and then gives the LLM the ability to call an appropriate function in order to complete its task. We can give it the option to call the function on its own if it wants to, or force it to call the function.

We will look at how this works, but first, let's notice a security warning mentioned in [OpenAI's guide on function calling](https://platform.openai.com/docs/guides/function-calling):

```
With this capability also comes potential risks. We strongly recommend building
in user confirmation flows before taking actions that impact the world on
behalf of users (sending an email, posting something online, making a purchase, etc).
```

This is a reminder to be wary of the security issues we discussed in Week 4 of your Deepskilling phase.

**Question to ponder**: Can you think of any potential risks from giving LLMs the ability to call functions?


## Model version and API version

At the time of writing (November 2023), OpenAI function calling is available only for the more recent models, `gpt-3.5-turbo-0613`, `gpt-4-0613` or newer ones.

Note that earlier in our code setup cell, we used an older API version:

    openai.api_version = '2023-05-15'

To enable function calling, you have to set your [API version](https://learn.microsoft.com/en-US/azure/ai-services/openai/reference#completions) to this or a newer version:

    openai.api_version = '2023-07-01-preview'

Refer to OpenAI's [API reference](https://learn.microsoft.com/en-US/azure/ai-services/openai/reference#example-response-2) -- scroll down to the parameter `function_call`.

Users are generally advised to use the latest non-preview API version, so you understand that function calling is still a relatively new and experimental feature.

**Additional note**

On 6 November 2023, [OpenAI announced](https://openai.com/blog/new-models-and-developer-products-announced-at-devday) newer models:

- gpt-3.5-turbo-1106
- gpt-4-1106-preview

This means even the `gpt-3.5-turbo-0613` and `gpt-4-0613` models are now considered older models, and they will be accessible only until 13 June 2024.


## Refresher on API

A **user interface** is a way for a human to communicate with a computer.

If instead of a human talking with a computer, you have two _computers_ talking with each other, we call it an **application programming interface** or API. An API is a way for computers to talk to each other.

APIs are structured in a specific and predictable format, so that computers won't get confused. Today, one of the most popular formats is **JSON**, which stands for JavaScript Object Notation.

Here is an example of what JSON looks like:

```json
{
  "first_name": "John",
  "last_name": "Tan",
  "is_alive": true,
  "age": 27,
  "address": {
    "street_address": "Blk 123 Ang Mo Kio Ave 1",
    "city": "Singapore",
    "state": "Singapore",
    "postal_code": "123456"
  },
  "phone_numbers": [
    {
      "type": "home",
      "number": "65 12345678"
    },
    {
      "type": "office",
      "number": "65 87654321"
    }
  ],
  "children": ["Ah Kow", "Ah Niao", "Ah Long"],
  "spouse": null
}
```

Notice the key-value pairs, and the structured hierarchy.

When computers communicate via an API, Computer A sends a **request** (usually a HTTP request) to Computer B which is listening all the time for requests. When Computer B receives the request, it sends a **response** back to Computer A.

It is common for developers to check the **API documentation** so that they know how to format the API requests and interpret the API responses.

APIs usually get updated over time, so there are different API versions available.


## Run setup code

Run this code cell. Notice we are setting the API_VERSION to a newer one.


`Remark`: You might need to restart the Colab session if subsequent error messages arise from the failure to import the updated version of OpenAI python library. To do so, follow the instructions:

- Select "Runtime"
- Select "Restart Session"
- rerun the Python code snippets below to install the specified version of `openai==1.12.0`


In [ ]:
# Install software packages:
%pip uninstall openai -y
%pip install openai==1.12.0
#%pip install openai==0.28.1
%pip install python-dotenv
%pip install requests

Found existing installation: openai 1.12.0
Uninstalling openai-1.12.0:
  Successfully uninstalled openai-1.12.0
  Using cached openai-1.12.0-py3-none-any.whl (226 kB)


In [4]:
%reload_ext autoreload
%autoreload 2
import openai
print(openai.VERSION)

0.28.1


In [ ]:
!pip show openai

Name: openai
Version: 1.12.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: llama-index


In [3]:
# %reload_ext autoreload
# %autoreload 2
# Import OpenAI package into Python:
from openai import AzureOpenAI

# Import environment packages:
from dotenv import load_dotenv
import os

# Import the json module:
import json

# Import display/formatting modules:
from IPython.display import Markdown, display

# Load .env file and get API key, API base and deployment name:
load_dotenv()
API_KEY = os.getenv("API_KEY")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")
AZURE_ENDPOINT = os.getenv("OPENAI_API_BASE")

# Use a newer API version which enables function calling:
API_VERSION = "2023-07-01-preview"

ImportError: cannot import name 'AzureOpenAI' from 'openai' (/usr/local/lib/python3.10/dist-packages/openai/__init__.py)

## OpenAI chat response without function calling

Before we start on the hands-on exercise, let's take a look at the chat response of the OpenAI API without any function calling. Run the code cell below which gives a normal greeting prompt to the LLM, and then let's see the output.


In [ ]:
# Instantiate an LLM:
llm = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)

# Manually create a message history to pass to the LLM.
# In this case, there is only 1 message in the list, which is our prompt:
messages = [
    {
        "role": "user",
        "content": "Hello how are you?"
    }
]

# Pass the prompt to the LLM to get a completion:
response = llm.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=messages,
)

# See the data type of the ChatCompletion object:
print(type(response))
# Print out the ChatCompletion object in readable JSON format:
print(response.model_dump_json(indent=2))

<class 'openai.types.chat.chat_completion.ChatCompletion'>
{
  "id": "chatcmpl-9FbMbWpawKR9ncP29bHVQYfKzGMq0",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! I'm an AI assistant, so I don't experience emotions, but I'm here to help you with any questions or tasks you have. How can I assist you today?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1713505573,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fin

**Mini quiz**: What is the name of the format of the above chat response object which we printed out?

**Hint**: We mentioned this in Knowledge Transfer 1. This format was launched by OpenAI in March 2023 to address the prompt injection problem.


## OpenAI's Moderation API

Now let's take a close look at the LLM's output.

Notice 2 parts in the above output:

- "prompt_filter_results"
- "content_filter_results"

These two filter results are automatically created as part of **OpenAI's Moderation API**, which helps developers identify and filter problematic content. This is done for both the **prompt** from the user (prompt_filter_results) and the **completion** from the LLM (content_filter_results).

Notice that the API is providing flags for specific areas:

- hate
- self_harm
- sexual
- violence

In the above prompt and completion, both were flagged as "false" (i.e. unflagged) and "safe" in each of the respective areas.

We won't go into how to make use of the Moderation API, but if you want to learn more, you can do a free online course by Andrew Ng's Deeplearning.ai:

- [Building Systems with the ChatGPT API](https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/) by Andrew Ng and Isa Fulford

Now let's do an example of function calling to see how it works.


## Hands-on dummy API function calling

For the purpose of understanding how function calling works, let's go through a simple example from the [OpenAI function calling guide](https://platform.openai.com/docs/guides/function-calling/function-calling).

**get_current_weather()**

For this example, we will create a 'dummy' API function named `get_current_weather()`. This function will pretend to be a real API function by returning a JSON output whenever it gets called. If this were a real API function, it would perform an actual API call to a remote server, get back some real-time data, and return that data in JSON format. But because this is a fake API function, the output will always come back as 72 degrees.

The dummy function is the first function that we define in the code cell below.

**run_conversation()**

We also define a second function `run_conversation()`, which does this:

- It sends a prompt to the LLM, asking: "What's the weather like in Boston?"

- We provide a Python dictionary which tells the LLM that there is a function `get_current_weather` which is available for calling. This function takes a location parameter, and an optional unit of measure, celsius or fahrenheit.

- We send the prompt to the LLM, along with the name of this function it can call, and we say that it has the option to call the function if it wants to, by setting an argument, `function_call="auto"`.

  There are other arguments available, such as explicitly telling the LLM to respond without function calling (`function_call="none"`) or forcing the LLM to always call a specified function (`function_call={"name": "my_function"}`). Refer to the [Azure OpenAI function calling reference](https://learn.microsoft.com/en-US/azure/ai-services/openai/reference#example-response-2) -- look at the `function_call` and `functions` parameters.

Look through the code below, run it, and then we will explain the output below.


In [ ]:
# Instantiate an LLM:
llm = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)

# Define dummy function, which is hardcoded to always return the same weather.
# In production, this could be your backend API or an external API.


def get_current_weather(location, unit="fahrenheit"):
    """ Get the current weather in a given location.
    Args:
        location (str): The city and state, e.g. San Francisco, CA.
        unit (str): "celsius" or "fahrenheit".

    Returns:
        str: JSON string.
    """
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation():
    """ Run an LLM conversation using function calling.

    Returns:
        response_message: OpenAI response message object.
    """
    # Step 1: Send the conversation and available functions to the LLM.
    # Define a prompt:
    messages = [
        {"role": "user", "content": "What's the weather like in Boston?"}]

    # Define a list of functions available to be called.
    # In this case, the list has only one function, get_current_weather, which
    # gets the weather information of a location in fahrenheit by default.
    # You can define more than one function.
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]

    # Send the prompt to the LLM and get back a response:
    response = llm.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        functions=functions,    # Pass in the function list which contains only
                                # one function.
        function_call="auto",   # "auto" means the LLM can decide to call the
                                # function if it wants to.
    )

    # Print out the full response object so that we can look at it.
    print(response.model_dump_json(indent=2))

    # Return the completion part of the response object, which we will print
    # after calling run_conversation:
    response_message = response.choices[0].message.model_dump_json(indent=2)

    return response_message


# Run the function to prompt the LLM:
first_response = run_conversation()

# Print the completion:
print('\nFirst response from LLM:')
print(first_response)

{
  "id": "chatcmpl-9FbTVusyR5UOatotklxR44e6VICNw",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      },
      "content_filter_results": {}
    }
  ],
  "created": 1713506001,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 82,
    "total_tokens": 100
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severit

Now that you have run the code, let's focus our attention on this part of the output:

```
      "finish_reason": "function_call",
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      ...
```

The "finish_reason" tells us that the LLM decided to make a function call, because we gave it the option to do so.

The "function_call" dict tells us the name of the function it called, which was `"get_current_weather"`, and the argument passed into the function, which was `"location": "Boston, MA"`. Notice that the LLM chose not to include the temperature unit argument, because it knew that it was optional. Only the location was required.

This is only step 1 of 4 steps. We paused here so that we can see what is happening so far.

In the code cell below, step 1 is the same, but now we run all the 4 steps at one go.

The remaining 3 steps will take the result after calling `get_current_weather`, give that weather data to the LLM, and then prompt the LLM to answer the original query, "What's the weather like in Boston?"


In [ ]:
def run_conversation():
    """ Run an LLM conversation using function calling.

    Returns:
        response_message: OpenAI response message object.
    """
    # Step 1: Send the conversation and available functions to the LLM.
    messages = [
        {"role": "user", "content": "What's the weather like in Boston?"}]

    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]

    # Send the prompt to the LLM and get back a response:
    response = llm.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        functions=functions,    # Pass in the function list which contains only
                                # one function.
        function_call="auto",   # "auto" means the LLM can decide to call the
                                # function if it wants to.
    )
    response_message = response.choices[0].message

    # Step 2: Check if the LLM wanted to call a function.
    # From the response_message a.k.a. the "message" dictionary, we attempt to
    # access the value of "function_call" key. If this key exists, the "if"
    # statement will be True, otherwise it will be None which is treated as False.
    if response.choices[0].message.function_call:

        # Step 3: Call the function.

        # Define the available functions:
        available_functions = {
            "get_current_weather": get_current_weather,
            # We define a key:value pair here, which is the name of the function
            # and the actual function object.
        }  # Only one function in this example, but you can have multiple.

        # Get the function name which the LLM wants to call:
        function_name = response_message.function_call.name
        # Find that function within the available functions:
        function_to_call = available_functions[function_name]
        # Get the function arguments which the LLM wants to use:
        function_args = json.loads(response_message.function_call.arguments)

        # Actual calling of the function:
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        # Let's see what is inside the function_response:
        print('Weather info from API:')
        print(function_response)
        print('\n')

        # Step 4: Send the function response to the LLM.
        # Extend conversation with function response:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        # Now send the second prompt to the LLM which includes the function response:
        second_response = llm.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
        )
        return second_response


# Run the LLM calling function and get a response:
response = run_conversation()
print(response.choices[0].model_dump_json(indent=2))

Weather info from API:
{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}


{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "message": {
    "content": "The weather in Boston is currently sunny and windy, with a temperature of 72 degrees Fahrenheit.",
    "role": "assistant",
    "function_call": null,
    "tool_calls": null
  },
  "content_filter_results": {
    "hate": {
      "filtered": false,
      "severity": "safe"
    },
    "self_harm": {
      "filtered": false,
      "severity": "safe"
    },
    "sexual": {
      "filtered": false,
      "severity": "safe"
    },
    "violence": {
      "filtered": false,
      "severity": "safe"
    }
  }
}


Now that you have run the code, let's go through the output. First, this is the output from the `get_current_weather` API function:

```
{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}
```

After calling the API function, we get back a JSON object, which is what you see here, containing the weather info about Boston.

Remember that the LLM chose to leave out the temperature unit when calling the API function. Notice that the "unit" is null, because the .get("unit") method returns None when the "unit" key was not found. The LLM nonetheless made an educated guess that the temperature was in Fahrenheit. If you like, you can try making the "unit" a required argument, and see how it changes the LLM's behaviour.

Also remember that the temperature and the forecast will always be the same no matter where the location because this is a fake API function.

This JSON output is stored in the variable `function_response`

In Step 4, `function_response` is given to the LLM by appending a dict to the `messages` list. The LLM is able to interpret the JSON data and use it as relevant context to inform its completion.

```
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
```

Finally, look at the completion from the LLM:

```
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees.",
    "role": "assistant",
    "function_call": null,
    "tool_calls": null
  },
```

The "finish_reason" tells us that the LLM stopped because it has finished answering the query about the weather.


## Hands-on real API function calling

In the following code cell, we demo a real API function call to retrieve jokes about Chuck Norris. The API server hosts a large collection of Chuck Norris jokes. All the jokes reference the popular view that Chuck Norris is a tough guy whose martial art skills are unbeatable. You supply a topic and the API will give back 3 jokes on that topic.

Notice that when we call the [API function](https://api.chucknorris.io) below by asking for jokes about a "fast food" topic, we get back 3 jokes in JSON format.


In [ ]:
# Import the 'requests' module for doing API calls:
import requests
# Import json module:
import json

# Define a function to retrieve Chuck Norris jokes:


def get_chuck_norris_joke_by_topic(topic="fast food"):
    """ Get Chuck Norris jokes about a given topic via an API call.

    Args:
        topic (str): The topic of the joke. Default is "fast food".

    Returns:
        jokes.text: A JSON string containing joke results.
    """
    api_url = 'https://api.chucknorris.io/jokes/search'

    params = {
        "query": topic,
    }

    jokes = requests.get(
        url=api_url,
        params=params,
    )
    return jokes.json()


# Test to check that the API works:
jokes = get_chuck_norris_joke_by_topic()
# Print out in JSON format:
json_str = json.dumps(jokes, indent=4)
print(json_str)

{
    "total": 3,
    "result": [
        {
            "categories": [
                "food"
            ],
            "created_at": "2020-01-05 13:42:18.823766",
            "icon_url": "https://assets.chucknorris.host/img/avatar/chuck-norris.png",
            "id": "Vnx6otXNQD6W04saS5qZjA",
            "updated_at": "2020-01-05 13:42:18.823766",
            "url": "https://api.chucknorris.io/jokes/Vnx6otXNQD6W04saS5qZjA",
            "value": "When Chuck Norris gets fast food, his order is ready before he walks in."
        },
        {
            "categories": [
                "food"
            ],
            "created_at": "2020-01-05 13:42:18.823766",
            "icon_url": "https://assets.chucknorris.host/img/avatar/chuck-norris.png",
            "id": "nIEoobK5RN-HHhbHbGf-_Q",
            "updated_at": "2020-01-05 13:42:18.823766",
            "url": "https://api.chucknorris.io/jokes/nIEoobK5RN-HHhbHbGf-_Q",
            "value": "Chuck Norris' idea of fast food is chasing 

Notice that the LLM knows how to interpret the JSON output from the API function. Even though the API returned 3 jokes, the LLM will pick only one of them to use for its natural language response.

In the code cell below, we demo another function which prompts the LLM to call the above API function.

The query is: "Tell me a Chuck Norris joke about weapons."

Run this code cell to see the output from the LLM:


In [ ]:
def run_chuck_norris():
    """ Run an LLM conversation using function calling.

    Returns:
        second_response: LLM response string.
    """
    # Step 1: Send the conversation and available functions to the LLM.
    messages = [
        {"role": "user", "content": "Tell me a Chuck Norris joke about weapons."}]

    functions = [
        {
            "name": "get_chuck_norris_joke_by_topic",
            "description": "Get Chuck Norris jokes by topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic of the joke. For example, fast food.",
                    },
                },
                "required": ["topic"],
            },
        }
    ]

    # Send the prompt to the LLM and get back a response:
    response = llm.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response.choices[0].message
    print(response_message)

    # Step 2: Check if the LLM wanted to call a function.
    if response.choices[0].message.function_call:

        # Step 3: Call the function.

        # Define the available functions:
        available_functions = {
            "get_chuck_norris_joke_by_topic": get_chuck_norris_joke_by_topic,
        }  # Only one function in this example, but you can have multiple.

        # Get the function name which the LLM wants to call:
        function_name = response_message.function_call.name
        print("function name:", function_name)
        # Find that function within the available functions:
        function_to_call = available_functions[function_name]
        # Get the function arguments which the LLM wants to use:
        function_args = json.loads(response_message.function_call.arguments)

        # Actual calling of the function:
        function_response = function_to_call(
            topic=function_args.get("topic"),
        )
        # To see what is inside the function_response:
        print('Joke info:')
        print("function response:", function_response)
        print('\n')
        print("function response result:",
              function_response["result"][0]["value"])

        # Step 4: Send the function response to the LLM.
        # Extend the conversation with function response:
        # messages.append(
        #     {
        #         "role": "function",
        #         "name": function_name,
        #         "content": function_response,
        #     }
        # )
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response["result"][0]["value"],
            }
        )
        print(messages)

        # Now send the second prompt to the LLM which includes the function response:
        second_response = llm.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
            stream=False,
        )
    return second_response


# Run the LLM calling function and get a response:
response = run_chuck_norris()
joke = response.choices[0].message.content
display(Markdown(f"{joke}"))

## Hands-on function calling exercise

Let's do a simple function calling exercise by using a free API which will check for you the current weather in Singapore.

If you hit this URL with your browser:

https://goweather.herokuapp.com/weather/Singapore

you will get a JSON response like this:

```json
{
    "temperature":"+30 °C",
    "wind":"6 km/h",
    "description":"Partly cloudy",
    "forecast":[
        {
            "day":"1",
            "temperature":"+26 °C",
            "wind":"18 km/h"
        },
        {
            "day":"2",
            "temperature":"+28 °C",
            "wind":"5 km/h"
        },
        {
            "day":"3",
            "temperature":" °C",
            "wind":"24 km/h"
        }
    ]
}
```

If you want to learn about this Weather API, you can [visit its GitHub](https://github.com/robertoduessmann/weather-api).

Alternatively, you may also visit the [data.gov.sg weather API](https://beta.data.gov.sg/collections/1456/datasets/d_91ffc58263cff535910c16a4166ccbc3/view) supplied by the National Environmental Agency (NEA).


**The first function to make the API call has already been written for you. Your task is to write the second function for prompting the LLM to make the function call.**


In [ ]:
# Import the 'requests' module for doing API calls:
import requests
# Import json module:
import json

# Define a function to retrieve current weather information in Singapore:


def get_sg_weather():
    """ Get the current weather information in Singapore.

    Returns:
        weather.text: A JSON string containing weather data.
    """
    api_url = 'https://goweather.herokuapp.com/weather/Singapore'
    # https://beta.data.gov.sg/collections/1456/datasets/d_91ffc58263cff535910c16a4166ccbc3/view
    # api_url = "https://api.data.gov.sg/v1/environment/2-hour-weather-forecast"

    weather = requests.get(
        url=api_url
    )
    return weather.json()


# Test to check that the API works:
response = get_sg_weather()
# Print out in JSON format:
json_str = json.dumps(response, indent=4)
print(json_str)

In [ ]:
# @title
# Import the 'requests' module for doing API calls:
import requests
# Import json module:
import json

# Define a function to retrieve current weather information in Singapore:


def get_sg_weather():
    """ Get the current weather information in Singapore.

    Returns:
        weather.text: A JSON string containing weather data.
    """
    # api_url = 'https://goweather.herokuapp.com/weather/Singapore'
    # https://beta.data.gov.sg/collections/1456/datasets/d_91ffc58263cff535910c16a4166ccbc3/view
    api_url = "https://api.data.gov.sg/v1/environment/2-hour-weather-forecast"

    weather = requests.get(
        url=api_url
    )
    return weather.json()


# Test to check that the API works:
response = get_sg_weather()
# Print out in JSON format:
json_str = json.dumps(response, indent=4)
print(json_str)

## Caveat: The dreaded rate limit

Sometimes, the above API function might fail with an error. This is likely because it is hosted on a free Heroku server which has a rate limit. You may have to wait a few minutes before you can call the API again.

If you just want to do the exercise without using a real API, you can use the following function instead:


In [ ]:
json_str = """
{
    "temperature":"+30 °C",
    "wind":"6 km/h",
    "description":"Partly cloudy",
    "forecast":[
        {
            "day":"1",
            "temperature":"+26 °C",
            "wind":"18 km/h"
        },
        {
            "day":"2",
            "temperature":"+28 °C",
            "wind":"5 km/h"
        },
        {
            "day":"3",
            "temperature":" °C",
            "wind":"24 km/h"
        }
    ]
}
"""


def get_fake_sg_weather():
    """ Fake API function to get the current weather information in Singapore.

    Returns:
        A JSON string containing hard-coded weather data.
    """
    return json_str


print(get_fake_sg_weather())

Now it's your turn. In the empty code cell below, write a a function containing the 4 steps get the LLM to make a function call to answer a query about what's the current weather in Singapore.


#### Code Activity Suggested Solution(s):

After completing the above code activity, you may click on "Show code" to reveal the various versions of suggested Code Activity Solutions below for comparison.

Alternatively, if you are stuck, you may also refer to the various versions of suggested Code Activity Solutions below for comparison.

In [ ]:
# Solution based on get_fake_sg_weather()
# @title
# Make sure to run the get_fake_sg_weather() code cell above before running this cell.
def get_fake_sg_weather():
    """ Fake API function to get the current weather information in Singapore.

    Returns:
        A JSON string containing hard-coded weather data.
    """
    return json_str
# Instantiate an LLM:
llm = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)

def run_sg_weather_api():
    """ Run an LLM conversation using function calling.

    Returns:
        second_response: LLM response string.
    """
    # Step 1: Send the conversation and available functions to the LLM.
    messages = [{"role": "user", "content": "What is the current weather in Singapore?"}]

    functions = [
        {
            "name": "get_fake_sg_weather",
            "description": "Get the current weather in Singapore",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        }
    ]

    # Send the prompt to the LLM and get back a response:
    response = llm.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response.choices[0].message

    # Step 2: Check if the LLM wanted to call a function.
    if response.choices[0].message.function_call:

        # Step 3: Call the function.

        # Define the available functions:
        available_functions = {
            "get_fake_sg_weather": get_fake_sg_weather,
        }
        # Get the function name which the LLM wants to call:
        function_name = response_message.function_call.name
        # Find that function within the available functions:
        function_to_call = available_functions[function_name]
        # Get the function arguments which the LLM wants to use:
        function_args = json.loads(response_message.function_call.arguments)

        # Actual calling of the function:
        function_response = function_to_call()
        # To see what is inside the function_response:
        # print('Function response:')
        # print(function_response)
        # print('\n')

        # Step 4: Send the function response to the LLM.
        # Extend the conversation with function response:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        # Now send the second prompt to the LLM which includes the function response:
        second_response = llm.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
        )
        return second_response

# Run the LLM calling function and get a response:
response = run_sg_weather_api()
llm_response = response.choices[0].message.content
display(Markdown(f"{llm_response}"))

In [ ]:
# @title
# Solution based on def get_sg_weather() on the heroku weather app API
# Instantiate an LLM:
llm = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)
def get_sg_weather():
    """ Get the current weather information in Singapore from Data.gov.sg API. """
    api_url = 'https://goweather.herokuapp.com/weather/Singapore'
    # api_url = "https://api.data.gov.sg/v1/environment/2-hour-weather-forecast"
    weather = requests.get(url=api_url)
    return json.dumps(weather.json())  # Make sure to return a JSON string

def run_get_sg_weather():
    """ Run an LLM conversation using function calling.

    Returns:
        second_response: LLM response string.
    """
    # Step 1: Send the conversation and available functions to the LLM.
    messages = [
        {"role": "user", "content": "Tell me information about the weather."}]

    functions = [
        {
            "name": "get_sg_weather",
            "description": "Obtain information about the weather.",
        }
    ]

    # Send the prompt to the LLM and get back a response:
    response = llm.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response.choices[0].message
    print(response_message)

    # Step 2: Check if the LLM wanted to call a function.
    if response.choices[0].message.function_call:

        # Step 3: Call the function.

        # Define the available functions:
        available_functions = {
            "get_sg_weather": get_sg_weather,
        }  # Only one function in this example, but you can have multiple.

        # Get the function name which the LLM wants to call:
        function_name = response_message.function_call.name
        print("function name:", function_name)
        # Find that function within the available functions:
        function_to_call = available_functions[function_name]
        # Get the function arguments which the LLM wants to use:
        function_args = json.loads(response_message.function_call.arguments)

        # Actual calling of the function:
        # function_response = function_to_call(
        #    topic=function_args.get("topic"),
        # )
        function_response = function_to_call()
        # To see what is inside the function_response:
        print('Weather info:')
        print("function response:", function_response)
        print('\n')
        #print("function response result:",
        #      function_response["result"][0]["value"])

        # Step 4: Send the function response to the LLM.
        # Extend the conversation with function response:
        # messages.append(
        #     {
        #         "role": "function",
        #         "name": function_name,
        #         "content": function_response,
        #     }
        # )
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response, # function_response["result"][0]["value"]
            }
        )
        print(messages)

        # Now send the second prompt to the LLM which includes the function response:
        second_response = llm.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
            stream=False,
        )
    return second_response


# Run the LLM calling function and get a response:
response = run_get_sg_weather()
weather_info = response.choices[0].message.content
display(Markdown(f"{weather_info}"))

In [ ]:
# @title
# # Solution based on def get_sg_weather() on the data.gov.sg NEA weather app API
# Instantiate an LLM:
llm = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)
def get_sg_weather():
    """ Get the current weather information in Singapore from Data.gov.sg API. """
    # api_url = 'https://goweather.herokuapp.com/weather/Singapore'
    api_url = "https://api.data.gov.sg/v1/environment/2-hour-weather-forecast"
    weather = requests.get(url=api_url)
    return json.dumps(weather.json())  # Make sure to return a JSON string

def run_get_sg_weather():
    """ Run an LLM conversation using function calling.

    Returns:
        second_response: LLM response string.
    """
    # Step 1: Send the conversation and available functions to the LLM.
    messages = [
        {"role": "user", "content": "Tell me information about the weather."}]

    functions = [
        {
            "name": "get_sg_weather",
            "description": "Obtain information about the weather.",
        }
    ]

    # Send the prompt to the LLM and get back a response:
    response = llm.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response.choices[0].message
    print(response_message)

    # Step 2: Check if the LLM wanted to call a function.
    if response.choices[0].message.function_call:

        # Step 3: Call the function.

        # Define the available functions:
        available_functions = {
            "get_sg_weather": get_sg_weather,
        }  # Only one function in this example, but you can have multiple.

        # Get the function name which the LLM wants to call:
        function_name = response_message.function_call.name
        print("function name:", function_name)
        # Find that function within the available functions:
        function_to_call = available_functions[function_name]
        # Get the function arguments which the LLM wants to use:
        function_args = json.loads(response_message.function_call.arguments)

        # Actual calling of the function:
        # function_response = function_to_call(
        #    topic=function_args.get("topic"),
        # )
        function_response = function_to_call()
        # To see what is inside the function_response:
        print('Weather info:')
        print("function response:", function_response)
        print('\n')
        #print("function response result:",
        #      function_response["result"][0]["value"])

        # Step 4: Send the function response to the LLM.
        # Extend the conversation with function response:
        # messages.append(
        #     {
        #         "role": "function",
        #         "name": function_name,
        #         "content": function_response,
        #     }
        # )
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response, # function_response["result"][0]["value"]
            }
        )
        print(messages)

        # Now send the second prompt to the LLM which includes the function response:
        second_response = llm.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
            stream=False,
        )
    return second_response


# Run the LLM calling function and get a response:
response = run_get_sg_weather()
weather_info = response.choices[0].message.content
display(Markdown(f"{weather_info}"))

## Note: Newer "tools" parameter (recently supported by Azure)

In early November 2023, OpenAI's [Create chat completion API](https://platform.openai.com/docs/api-reference/chat/create) deprecated the "functions" and "function_call" parameters, which are now superceded respectively by the new "tools" and "tool_choice" parameters.

This means we are not supposed to use "functions" and "function_call" parameters anymore, like we did in the previous coding exercise.

However, it seems that the **Microsoft Azure OpenAI service** is lagging behind **OpenAI**, because the "tools" and "tool_choice" parameters generate an error if you use them at this stage.

This may change in the near future, when Azure catches up with OpenAI. So you may find that the "functions" and "function_call" parameters may stop working at some point.

Below, we share some sample code using the newer parameters of the chat.completions.create() function -- in case you need it in future for Azure, or if you use OpenAI API instead of Azure OpenAI API. In the error message below the code cell, you can see that Azure OpenAI does not recognize the `tool_choice` and `tools` arguments.

Note that even using chat.completions.create() to do function calling may get deprecated in future. The latest way to do function calling is by using the Assistants API which has a function to do function calling: [beta.assistants.create()](https://platform.openai.com/docs/assistants/tools/function-calling). The Assistants API is still in beta now, but once it becomes the standard, you may need to update your code.


https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=python


In [ ]:
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("API_KEY"),
    api_version="2024-03-01-preview"
)


# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [
        {"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        # extend conversation with assistant's reply
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


response = run_conversation()
print(response.choices[0].model_dump_json(indent=2))

---


# Further learning

**OpenAI function calling and more**

- [OpenAI's announcement about function calling](https://openai.com/blog/function-calling-and-other-api-updates) (June 2023)
- [OpenAI's function calling guide](https://platform.openai.com/docs/guides/gpt/function-calling)
- [Microsoft Azure OpenAI's API reference](https://learn.microsoft.com/en-US/azure/ai-services/openai/reference)
- Deeplearning.ai short course: [Building Systems with the ChatGPT API](https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/) by Andrew Ng and Isa Fulford

**Function calling with LangChain**

- [A Complete LangChain Guide](https://nanonets.com/blog/langchain/) by Karan Kalra

**Retrieval Augmented Generation**

- [Evaluate RAG with LlamaIndex](https://github.com/openai/openai-cookbook/blob/main/examples/evaluation/Evaluate_RAG_with_LlamaIndex.ipynb)

**Low code tools**

- [Getting started with Vertex AI Generative AI Studio's User Interface](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/getting-started/intro_generative_ai_studio.md)
